In [3]:
# importing libraries

import torch, torchvision, os, PIL, pdb
from torch import nn 
from torch.utils.data import Dataset
from torch.utils.data import dataloader
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm
from PIL import Image
import matplotlib.pyplot as plt 

def show(tensor, num=25, wandb=0, name=''):
    data = tensor.detach().cpu()
    grid = make_grid(data[:num], nrow = 5).permute(1,2,0)

    if(wandb==1 and wandbact==1):
        wandb.log({name:wandb.Image(grid.numpy().clip(0,1))})

    plt.imshow(grid.clip(0,1))
    plt.show()


n_epochs = 500
batch_size = 128
lr = 1e-4
z_dim = 200
device = 'cuda'


cur_step = 0
crit_cycles = 5
gen_losses = []
crit_losses = []

show_stemp = 35
save_step = 35 # checkpoint

wandbact= 1













/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#!pip3 install wandb -qqq


In [5]:
import wandb
wandb.login(key='2290f8de6f8ae3c7391ad9cb96f5a7b9fd780daf')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rahulsm-27. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/rahulmahajan/.netrc


True

In [6]:
%%capture
experiment_name = wandb.util.generate_id()
myrun = wandb.init(
    project = 'wgan',
    group = experiment_name,
    config={"optimzier":"adam", "model":"wgan_gp"}

)

config = wandb.config



In [7]:
print(experiment_name)

zlx7c5p7


In [8]:
# Generator

# Crtic

class Generator(nn.Module):
    def __init__(self,z_dim=64,d_dim=16):
        super(Generator,self).__init__()
        self.z_dim = z_dim
        # we begin with 1*1 image with z_dim number of shcannels
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim,d_dim*32,4,1,0), ## 4 * 4 (ch : 200,512)
            nn.BatchNorm2d(d_dim*32 ) ,
            nn.RelU(True),


            nn.ConvTranspose2d(d_dim*32,d_dim*16,4,1,0), ## 8*8
            nn.BatchNorm2d(d_dim*32 ) ,
            nn.RelU(True),


            nn.ConvTranspose2d(d_dim*16,d_dim*8,4,1,0), ## 16*16
            nn.BatchNorm2d(d_dim*32 ) ,
            nn.RelU(True),


            nn.ConvTranspose2d(d_dim*8,d_dim*4,4,1,0), ## 32*32
            nn.BatchNorm2d(d_dim*32 ) ,
            nn.RelU(True),

            nn.ConvTranspose2d(d_dim*4,d_dim*2,4,1,0), ## 64*64
            nn.BatchNorm2d(d_dim*32 ) ,
            nn.RelU(True),

            nn.ConvTranspose2d(d_dim*2,3,4,1,0), ## 128*128
            nn.Tanh(),
                                
                                 
                                 )
        

    def forward(self, noise):
        x = noise.view(len(noise), self.z_dim , 1 , 1)
        return self.gen(x)    
    

    def gen_noise(num,z_dim, device='cuda'):
        return torch.randn(num,z_dim,device=device)

